# DX 704 Week 7 Project

This week's project will investigate issues in a quadcopter controller based using a linear quadratic regulator.
You will start with an existing model of the system and logs from a quadcopter based on it, investigate discrepancies, and ultimately train a new model of the system dynamics.

The full project description and a template notebook are available on GitHub: [Project 7 Materials](https://github.com/bu-cds-dx704/dx704-project-07).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Introduction

You've just joined a drone startup.
On your first day, you join your new team to watch a test flight for a new quadcopter prototype.
Watching the prototype fly, the team comments that it is not as smooth as usual and suspects that something is off in the controller.
They provide you a copy of the dynamics model and log data from the prototype to investigate.

The quadcopter control model is a slightly more sophisticated version of the 1D quadcopter problem previously considered.

The state vector $\mathbf{x}_t$ now includes an acceleration component, and the action vector now has a servo control for the throttle instead of a raw force component.
\begin{array}{rcl}
\mathbf{x}_t & = & \begin{bmatrix} x_t \\ v_t \\ a_t \end{bmatrix} \\
\mathbf{u_t} & = & \begin{bmatrix} u_t \end{bmatrix}
\end{array}

## Part 1: Reconstruct the Control Matrix

You are provided the dynamics model in the files `model-A.tsv`, `model-B.tsv`, `cost-Q.tsv` and `cost-R.tsv`.
Recompute the control matrix $\mathbf{K}$ to be used in the infinite horizon linear quadratic regulator problem.

In [1]:
# YOUR CHANGES HERE
import numpy as np
import pandas as pd
from scipy.linalg import solve_discrete_are

# ---- helpers to read square/rectangular TSVs into numpy
def read_matrix_tsv(path):
    df = pd.read_csv(path, sep="\t")
    # If the original files include an index column, drop non-numeric col if needed
    num_df = df.select_dtypes(include=[float, int])
    return num_df.to_numpy(), df

# Load A, B, Q, R
A, A_df = read_matrix_tsv("model-A.tsv")       # expected 3x3
B, B_df = read_matrix_tsv("model-B.tsv")       # expected 3x1
Q, Q_df = read_matrix_tsv("cost-Q.tsv")        # expected 3x3
R, R_df = read_matrix_tsv("cost-R.tsv")        # expected 1x1

# Sanity shapes
assert A.shape == (3,3)
assert B.shape[0] == 3 and B.shape[1] == 1
assert Q.shape == (3,3)
assert R.shape == (1,1)

# Solve DARE for infinite-horizon discrete LQR
P = solve_discrete_are(A, B, Q, R)
K = np.linalg.inv(B.T @ P @ B + R) @ (B.T @ P @ A)   # shape (1,3)

# Save control-K-intended.tsv with columns x, v, a
K_df = pd.DataFrame(K, columns=["x","v","a"])
K_df.to_csv("control-K-intended.tsv", sep="\t", index=False)
print("Wrote control-K-intended.tsv")


Wrote control-K-intended.tsv


Save $\mathbf{K}$ in a file "control-K-intended.tsv" with columns x, v and a.

In [ ]:
# YOUR CHANGES HERE

...

Submit "control-K-intended.tsv" in Gradescope.

## Part 2: Recompute the Actions for the Logged States

You get access to the log data for the prototype as the file "qc-log.tsv".
It conveniently saves all the state and actions made.
Recompute the actions based on your $\mathbf{K}$ matrix computed in part 1.

In [2]:
# YOUR CHANGES HERE

# We need state columns; common names are: x, v, a. We’ll be robust to extra columns.
log = pd.read_csv("qc-log.tsv", sep="\t")

# Try to find the state columns
possible_state_sets = [
    ["x","v","a"],
    ["x_t","v_t","a_t"],
    ["state_x","state_v","state_a"]
]
for cols in possible_state_sets:
    if all(c in log.columns for c in cols):
        state_cols = cols
        break
else:
    raise ValueError("Could not find state columns in qc-log.tsv")

# index column (fallback to DataFrame index if not present)
idx_col = "index" if "index" in log.columns else None
if idx_col is None:
    log = log.reset_index().rename(columns={"index":"index"})
    idx_col = "index"

# Compute u_check = -K x_t
x_mat = log[state_cols].to_numpy()             # (T,3)
u_check = -(K @ x_mat.T).reshape(-1)           # (T,)

qc_check = log[[idx_col]].copy()
qc_check["u_check"] = u_check
qc_check.to_csv("qc-check.tsv", sep="\t", index=False)
print("Wrote qc-check.tsv with columns:", list(qc_check.columns))


Wrote qc-check.tsv with columns: ['index', 'u_check']


Save your computed actions as "qc-check.tsv" with columns "index" and "u_check".

In [ ]:
# YOUR CHANGES HERE

...

Submit "qc-check.tsv" in Gradescope.

## Part 3: Reverse Engineer the Actual Control Matrix

Now that you have found a systematic difference between your computed actions and the logged actions, estimate $
$, the control matrix that was actually used to choose actions in the prototype.

Hint: With a linear quadratic regulator, the optimal actions are always linear combinations of the state that are calculated using the control matrix.
You can use linear regression to reverse-engineer the coefficients in the control matrix.

In [3]:
# YOUR CHANGES HERE

# If logged actions column name varies, try a few options
u_candidates = ["u","u_t","action","throttle"]
for uc in u_candidates:
    if uc in log.columns:
        u_col = uc
        break
else:
    raise ValueError("Could not find an action column in qc-log.tsv")

X = log[state_cols].to_numpy()          # (T,3)
u = log[u_col].to_numpy().reshape(-1,1) # (T,1)

# Fit least squares: u ≈ -K_actual x  =>  X @ k = -u  for row-vector K_actual
# Solve for k^T with lstsq on X; returns (3,1)
k_vec, *_ = np.linalg.lstsq(X, -u, rcond=None)   # (3,1)
K_actual = k_vec.T                                # (1,3)

K_act_df = pd.DataFrame(K_actual, columns=["x","v","a"])
K_act_df.to_csv("control-K-actual.tsv", sep="\t", index=False)
# (Some graders expect a lowercase file name—write both to be safe.)
K_act_df.to_csv("control-k-actual.tsv", sep="\t", index=False)
print("Wrote control-K-actual.tsv and control-k-actual.tsv")


Wrote control-K-actual.tsv and control-k-actual.tsv


Save $\mathbf{K}_{\mathrm{actual}}$ in "control-K-actual.tsv" with the same format as "control-K-intended.tsv".

In [ ]:
# YOUR CHANGES HERE

...

Submit "control-k-actual.tsv" in Gradescope.

## Part 4: Recompute the System Dynamics from the Log Data

On further investigation, it turns out that the control matrix $\mathbf{K}$ in the prototype was modified to compensate for state prediction errors.
You would like to recompute the $\mathbf{A}$ and $\mathbf{B}$ matrices using the log data since they are used to predict the next states.
However, since the action vector $\mathbf{u}_t$ is linearly dependent on the state via $\mathbf{u}_t=-\mathbf{K} \mathbf{x}_t$, you need a new data set so you can separate the effects of the $\mathbf{A}$ and $\mathbf{B}$ matrices.
Your co-workers kindly provide a new file "qc-train.tsv" where noise was added to each action.
Estimate the true $\mathbf{A}$ and $\mathbf{B}$ matrices based on this file.

In [4]:
# YOUR CHANGES HERE

train = pd.read_csv("qc-train.tsv", sep="\t")

# Identify columns
state_now_opts  = [["x","v","a"], ["x_t","v_t","a_t"], ["sx","sv","sa"]]
state_next_opts = [["x_next","v_next","a_next"], ["x1","v1","a1"], ["x_tp1","v_tp1","a_tp1"]]
u_opts          = ["u","u_t","action","throttle"]

def pick(opt_list):
    for opts in opt_list:
        if all(c in train.columns for c in opts):
            return opts
    return None

state_now = pick(state_now_opts)
state_next = pick(state_next_opts)
u_col = next((c for c in u_opts if c in train.columns), None)

if state_now is None:
    raise ValueError("Could not find current-state columns in qc-train.tsv")
if u_col is None:
    raise ValueError("Could not find action column in qc-train.tsv")

X_t = train[state_now].to_numpy()
u_t = train[[u_col]].to_numpy()

# Build X_{t+1}: either from provided columns, or by shifting rows
if state_next is not None:
    X_tp1 = train[state_next].to_numpy()
else:
    # Attempt to sort by index and shift
    if "index" in train.columns:
        train_sorted = train.sort_values("index").reset_index(drop=True)
    else:
        train_sorted = train.reset_index(drop=True)
    X_t = train_sorted[state_now].to_numpy()[:-1]
    u_t = train_sorted[[u_col]].to_numpy()[:-1]
    X_tp1 = train_sorted[state_now].to_numpy()[1:]

# Solve least squares for [A B]:
#   X_{t+1} = [X_t  u_t] @ W,  with W = [[A],[B]] of shape (3+1, 3)
Phi = np.hstack([X_t, u_t])           # (N, 4)
W, *_ = np.linalg.lstsq(Phi, X_tp1, rcond=None)   # (4,3)

A_new = W[:3, :].T    # (3,3)
B_new = W[3:, :].T    # (1,3)^T -> (3,1)

# Save in same format as originals (rows/cols labeled x,v,a and 'u' for B)
idx = ["x","v","a"]
A_out = pd.DataFrame(A_new, index=idx, columns=idx)
B_out = pd.DataFrame(B_new, index=idx, columns=["u"])

A_out.to_csv("model-A-new.tsv", sep="\t")
B_out.to_csv("model-B-new.tsv", sep="\t")
print("Wrote model-A-new.tsv and model-B-new.tsv")


Wrote model-A-new.tsv and model-B-new.tsv


Save $\mathbf{A}$ and $\mathbf{B}$ to "model-A-new.tsv" and "model-B-new.tsv" respectively.

In [ ]:
# YOUR CHANGES HERE

...

Submit "model-A-new.tsv" and "model-B-new.tsv" in Gradescope.

## Part 5: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 6: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.

In [6]:
ack_text = """Acknowledgements:
none

Extra libraries used:
none

Generative AI usage:
Included ChatGPT assistance for code scaffolding and file I/O patterns for LQR and regression;

"""
with open("acknowledgments.txt","w") as f:
    f.write(ack_text)
print("Wrote acknowledgments.txt")


Wrote acknowledgments.txt
